In [22]:
import numpy as np
from collections import defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.spatial import distance
import re, sys, os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from IPython.display import display
import seaborn.apionly as sns
plt.style.use('ggplot')
sns.set_style('ticks')
import matplotlib.ticker as ticker
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from torchvision import datasets
from IPython.display import Image
import h5py
import numpy
import theano
import theano.tensor as T
from theano import pp

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.optim.lr_scheduler import ReduceLROnPlateau

from skimage import io
import argparse
import scipy.io as sio

# Enable inline plotting  
%matplotlib inline

In [23]:
sys.path.append("/Users/mahzadkhoshlessan/Desktop/Machine-Learning/Intro-to-Deep-Learning/Final_Project/")

In [24]:
from CNN_Model import project_01, normalize_im, NeuralNet, L1L2loss, weight_init, Checkpoint

In [25]:
datafile = 'ArtificialDataset.tif'

In [26]:
Images = io.imread(datafile)
(K, M, N) = Images.shape

In [27]:
upsampling_factor = 8

In [28]:
# upsampling using a simple nearest neighbor interp.
Images_upsampled = np.zeros((K, M*upsampling_factor, N*upsampling_factor))
for i in range(Images.shape[0]):
    Images_upsampled[i,:,:] = np.kron(Images[i,:,:], np.ones((upsampling_factor,upsampling_factor)))   
Images = Images_upsampled

In [12]:
# upsampled frames dimensions
(K, M, N) = Images.shape

In [21]:
optimizer = optim.Adam(myModel.parameters(), lr=0.001)
criterion = L1L2loss((1, M, N))

In [14]:
myModel = NeuralNet()

In [19]:
weights_file = os.path.abspath(os.path.normpath(os.path.join(os.getcwd(),'Checkpointer.h5')))

In [ ]:
checkpoint = torch.load(weights_file)

In [ ]:
myModel.load_state_dict(checkpoint['state_dict'])
myModel.eval() 

In [ ]:
# load mean and std
matfile = sio.loadmat(meanstd_file)
test_mean = np.array(matfile['mean_test'])
test_std = np.array(matfile['std_test'])   

# Setting type
Images = Images.astype('float32')

In [ ]:
# Normalize each sample by it's own mean and std
Images_norm = np.zeros(Images.shape,dtype=np.float32)
for i in range(Images.shape[0]):
    Images_norm[i,:,:] = project_01(Images[i,:,:])
    Images_norm[i,:,:] = normalize_im(Images_norm[i,:,:], test_mean, test_std)

# Reshaping
Images_norm = np.expand_dims(Images_norm,axis=3)    

In [ ]:
# Make a prediction
predicted_density = myModel(Images)

In [ ]:
# threshold negative values
predicted_density[predicted_density < 0] = 0

# resulting sum images
WideField = np.squeeze(np.sum(Images_norm, axis=0))
Recovery = np.squeeze(np.sum(predicted_density, axis=0))

In [ ]:
# Look at the sum image
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, sharex=True)
ax1.imshow(WideField)
ax1.set_title('Wide Field')
ax2.imshow(Recovery)
ax2.set_title('Sum of Predictions')
f.subplots_adjust(hspace=0)
plt.show()

In [ ]:
# Save predictions to a matfile to open later in matlab
mdict = {"Recovery": Recovery}
sio.savemat(savename, mdict)

In [ ]:
# save predicted density in each frame for debugging purposes
mdict = {"Predictions": predicted_density}
sio.savemat(savename + '_predictions.mat', mdict)